In [1]:

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import random

In [2]:
# Function to get error code based on the error message
def get_error_code(error_message):
    if 'It looks like you’re a UScellular billed customer. To utilize Quick Pay for our billed customers' in error_message:
        return 1
    elif 'Phone number not valid' in error_message:
        return 3
    elif 'This number is no longer in service' in error_message:
        return 2
    elif 'Please verify you are a human before continuing' in error_message:
        return 4    
    elif 'No errors, it is a prepaid number' in error_message:
        return 0
    else:
        return -1  # Unknown error message

In [3]:
# Read phone numbers from file and remove country code
with open('test_panel_usus.txt', 'r') as file:
    phone_numbers = [line.strip()[1:] if line.strip().startswith('1') else line.strip() for line in file][95:]
# phone_numbers[0]="9192228197"
phone_numbers[0]

'5738812327'

In [4]:
options = webdriver.ChromeOptions()
options.headless = True

results = []
consecutive_error_count = 0
current_index = 0  # Initialize the index to keep track of the current phone number

def init_driver():
    """Initialize the Chrome WebDriver."""
    driver = webdriver.Chrome(options=options)
    driver.get('https://prepaid.uscellular.com/refill/dashboard')
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))
    ).click()
    return driver

In [6]:
# Initialize the WebDriver for the first time
driver = init_driver()
# Tracking variables
results = []
consecutive_error_count = 0
current_index = 0  # Initialize the index to keep track of the current phone number

prepaid_xpath = '<path d="M9 16.17L4.83 12l-1.42 1.41L9 19 21 7l-1.41-1.41z"></path>'
error_img_xpath = '/html/body/div[1]/main/div[1]/div/div[2]/div/div[2]/div[1]/div/div[2]/div/div/div/div/form/div[1]/div[1]/div[1]/div/div[2]/div/div/div/img'
error_message_xpath = "/html/body/div[1]/main/div[1]/div/div[2]/div/div[2]/div[1]/div/div[2]/div/div/div/div/form/div[1]/div[1]/div[2]"

# Start processing
while current_index < len(phone_numbers):
        
    #phone_numbers[current_index]
    phone_number = '5738812327'#phone_numbers[current_index]
    print(f"Processing phone number {phone_number} at index {current_index}")

    # Phone number input field
    phone_input_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "phone"))
    )
    phone_input_field.click()  # Focus on the input field

    # Clear the input field
    phone_input_field.send_keys(Keys.CONTROL + "a")
    phone_input_field.send_keys(Keys.DELETE)
    
    for char in phone_number:
        phone_input_field.send_keys(char)
        time.sleep(random.uniform(0.1, 1))  # Small delay between each character          
    time.sleep(1)
    
    try:
        # Input phone number
        elements = EC.visibility_of_element_located((By.XPATH, prepaid_xpath))
        try:
            # WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, error_message_xpath)))
            # WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, error_img_xpath)))
            error_message_element = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, error_message_xpath))
            )
            error_message_text = error_message_element.text.strip()
            error_code = get_error_code(error_message_text)  # Update error code based on message
            results.append((phone_number, error_code, error_message_text))
            print(phone_number, error_code, error_message_text)
            
            current_index += 1
            consecutive_error_count += 1
            if consecutive_error_count == 3:
                print("2")
                driver.quit()
                time.sleep(random.uniform(1, 2))
                driver = init_driver()
                consecutive_error_count = 0
            continue
        except TimeoutException:
            print("A timeout occurred while waiting for the element.")
            driver.quit()
            time.sleep(random.uniform(2, 5))
            driver = init_driver()  # Re-initialize the WebDriver
            continue

        except:
            error_code = 0
            print("Prepaid:")
            error_code = 0
            error_message = 'Prepaid'
            results.append((phone_number, error_code, error_message))
            print(phone_number, error_code, error_message)
            current_index += 1 
            continue          



    except:
        continue

# Clean up
driver.quit()

Processing phone number 5738812327 at index 0
A timeout occurred while waiting for the element.
Processing phone number 5738812327 at index 0


MaxRetryError: HTTPConnectionPool(host='localhost', port=63914): Max retries exceeded with url: /session/acf857e41dcb5a83b1c5ebe821ad87d8/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000019B2D139A60>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [94]:
# Convert the results to a DataFrame 
df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Code','Error Message'])
print(df_results)

Empty DataFrame
Columns: [Phone Number, Error Code, Error Message]
Index: []


In [95]:
import os

csv_file_path = 'uscs_query_result.csv'

df_results_error = pd.DataFrame(results, columns=['Phone Number', 'Error Code','Error Message'])
df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Code','Error Message'])

if os.path.exists(csv_file_path):
    df_existing = pd.read_csv(csv_file_path)
    
    last_index = df_existing.index[-1] if not df_existing.empty else -1

    df_results.index = range(last_index + 1, last_index + 1 + len(df_results))
    
    # Append the new results to the existing DataFrame
    df_final = df_existing.append(df_results)
    df_final = df_final.iloc[last_index + 1:,1:]
else:
    df_final = df_results
    df_final.index = range(len(df_final))

# Output to CSV, without overwriting existing data and with continuous indexing
df_final.to_csv(csv_file_path, index_label='Index', mode='w' if not os.path.exists(csv_file_path) else 'a', header=not os.path.exists(csv_file_path))

print(f"Results have been appended to {csv_file_path}")


Results have been appended to uscs_query_result.csv


C:\Users\ericluo\AppData\Local\Temp\ipykernel_24052\2492793956.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_existing.append(df_results)
